In [1]:
import pvlib
import json
from collections import Counter

In [2]:
inverter = pvlib.pvsystem.retrieve_sam(
            "CECinverter", path="CEC Inverters.csv"
        )

In [3]:
names_split = [name.split("_") for name in inverter.keys()]
occurences = Counter([part for parts_list in names_split for part in parts_list])

In [4]:
inverter_database = {}
for i, name_split in enumerate(names_split):
    name_split_no_empty = [part for part in name_split if part != ""]
    manufacturer = name_split_no_empty[0]
    series = ""
    model = name_split_no_empty[-1]
    
    left_over_parts =  name_split_no_empty[1:-1]
    while len(left_over_parts) > 0:
        if len(left_over_parts) == 1 and series == "": # series
            series = left_over_parts.pop(0)
        elif occurences[left_over_parts[0]] > 1 and not any([c.isdigit() for c in left_over_parts[0]]): #manufacturer
            manufacturer += " " + left_over_parts.pop(0)
        elif occurences[left_over_parts[-1]] == 1: # model
            model = left_over_parts.pop(-1) + " " + model
        else: # series
            if series != "":
                series += " " + left_over_parts.pop(0)
            else:
                series = left_over_parts.pop(0)
    if manufacturer not in inverter_database:
        inverter_database[manufacturer] = {}
    if series not in inverter_database[manufacturer]:
        inverter_database[manufacturer][series] = {}
    inverter_database[manufacturer][series][model] = {k:v if v != "NaN" else None for k,v in inverter["_".join(name_split)].to_dict().items()}

In [5]:
inverter_database["SMA America US"]["SB10000TL"]["208V"]

{'Vac': '208',
 'Pso': 20.301853,
 'Paco': 10100.0,
 'Pdco': 10452.24707,
 'Vdco': 345.0,
 'C0': -2.961646e-06,
 'C1': -4.789613e-06,
 'C2': -0.000714,
 'C3': -0.001077,
 'Pnt': 3.03,
 'Vdcmax': 480.0,
 'Idcmax': 30.296368,
 'Mppt_low': 100.0,
 'Mppt_high': 480.0,
 'CEC_Date': nan,
 'CEC_Type': 'Utility Interactive'}

In [6]:
with open("G:/projects/solar-power-estimator/data/inverter_database.json", "w") as f:
    json.dump(inverter_database, f, indent=2)